In [ ]:
!pip install -U transformers datasets openai lancedb FlagEmbedding "tantivy>=0.20.1" -qq

# NOTE: If there is an import error, restart and run the notebook again

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.7/226.7 kB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.0/113.0 kB 15.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 26.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.6/21.6 MB 36.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 30.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.8/132.8 kB 568.9 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 k

In [ ]:
import os
import lancedb
import re
import pandas as pd
import random

from datasets import load_dataset

import torch
import gc

import lance


import os

import lancedb
import openai
from lancedb.embeddings import get_registry
from lancedb.pydantic import LanceModel, Vector


os.environ["OPENAI_API_KEY"] = "sk-......"


embeddings = get_registry().get("openai").create()

# Load `Chunks` of data from [BeIR Dataset](https://huggingface.co/datasets/BeIR/scidocs)

Note: This is a dataset built specially for retrieval tasks to see how good your search is working

In [ ]:
queries = load_dataset("BeIR/scidocs", "queries")["queries"].to_pandas()
full_docs = load_dataset('BeIR/scidocs', 'corpus')["corpus"].to_pandas().dropna(subset = "text")

docs = full_docs.head(64) # just random samples for faster embed demo
docs["num_words"] = docs["text"].apply(lambda x: len(x.split())) # Insert some Metadata for a more "HYBRID" search
docs.sample(5)

<ipython-input-4-752074abc1be>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  docs["num_words"] = docs["text"].apply(lambda x: len(x.split())) # Insert some Metadata for a more "HYBRID" search


,_id,title,text,num_words
35,9ea16bc34448ca9d713f4501f1a6215a26746372,A survey of software testing practices in alberta,Software organizations have typically de-empha...,236
42,9d1940f843c448cc378214ff6bad3c1279b1911a,Shape-aware Instance Segmentation,We address the problem of instance-level seman...,187
31,4b53f660eb6cfe9180f9e609ad94df8606724a3d,Text mining of news-headlines for FOREX market...,In this paper a novel approach is proposed to ...,621
21,b579366db457216b0548220bf369ab9eb183a0cc,An analysis on the significance of ticket anal...,Software even though intangible should undergo...,232
29,8e508720cdb495b7821bf6e43c740eeb5f3a444a,Learning Scalable Deep Kernels with Recurrent\...,"Many applications in speech, robotics, finance...",138


# Build New Table

In [ ]:
!rm -rf ./db

In [ ]:
class Documents(LanceModel):
    vector: Vector(embeddings.ndims()) = embeddings.VectorField()
    text: str = embeddings.SourceField()
    title: str
    num_words: int

data = docs.apply(lambda row: {"title":row["title"], "text":row["text"], "num_words":row["num_words"]}, axis = 1).values.tolist()

db = lancedb.connect("./db")
table = db.create_table("documents", schema=Documents)

table.add(data) # ingest docs with auto-vectorization
table.create_fts_index("text") # Create a fts index before the hybrid search

In [ ]:
table.search("To confuse the AI and DNN embedding, let's put random terms from other sentences- automation training test memory?",
             query_type="fts").limit(5).to_pandas()

,vector,text,title,num_words,score
0,"[-0.0123116635, -0.003029472, 0.022851666, -0....","Sociological and technical difficulties, such ...",Hipikat: a project memory for software develop...,210,6.811104
1,"[-0.013957698, 0.02428029, 0.0018158188, -0.03...",Hierarchical Pitman-Yor Process priors are com...,Compressed Nonparametric Language Modelling,134,6.782823
2,"[-0.035857946, -0.00020074612, 0.009314451, -0...",We introduce a long short-term memory recurren...,Speech-driven 3 D Facial Animation with Implic...,171,6.157275
3,"[-0.03081366, -0.0026948636, 0.010220175, -0.0...",This report summarizes the objectives and eval...,SemEval-2015 Task 11: Sentiment Analysis of Fi...,112,6.050443
4,"[0.0012136719, -0.022606706, 0.0142465895, -0....",Software organizations have typically de-empha...,A survey of software testing practices in alberta,236,6.037307


In [ ]:
table.search("To confuse the AI and DNN embedding, let's put random terms from other sentences- automation training test memory?", query_type="vector").\
            limit(10).\
            to_pandas()

,vector,text,title,num_words,_distance
0,"[-0.029037403, -0.0019001433, 0.01014971, -0.0...",Classifying short texts to one category or clu...,Using deep learning for short text understanding,187,0.354801
1,"[-0.013545574, -0.0019434954, 0.009623939, -0....",a r t i c l e i n f o a b s t r a c t We prese...,"BabelNet: The automatic construction, evaluati...",133,0.402469
2,"[-0.027462298, -0.00018946378, 0.017612722, -0...",We introduce a technique for augmenting neural...,Deep Voice 2 : Multi-Speaker Neural Text-to-Sp...,151,0.417934
3,"[-0.035857946, -0.00020074612, 0.009314451, -0...",We introduce a long short-term memory recurren...,Speech-driven 3 D Facial Animation with Implic...,171,0.421723
4,"[-0.010440272, -0.0065525724, 0.018469144, -0....",The presented work aims at generating a system...,BCSAT : A Benchmark Corpus for Sentiment Analy...,126,0.430030
5,"[-0.037621364, 0.011013092, 0.02912708, -0.034...",This paper proposes quiz-style information pre...,Effects of quiz-style information presentation...,141,0.441784
6,"[-0.03081366, -0.0026948636, 0.010220175, -0.0...",This report summarizes the objectives and eval...,SemEval-2015 Task 11: Sentiment Analysis of Fi...,112,0.445057
7,"[-0.009847619, 0.007252175, 0.00984077, -0.031...",This paper presents a simple unsupervised lear...,Thumbs Up or Thumbs Down? Semantic Orientation...,163,0.448393
8,"[-0.02750998, 0.017241215, 0.016468005, -0.014...","With the development of online advertisements,...",Clickbait Convolutional Neural Network,160,0.448454
9,"[-0.016168512, 0.010419708, 0.029975908, -0.01...",Theories on the functions of the hippocampal s...,"Memory, navigation and theta rhythm in the hip...",130,0.453170


## Perform inbuilt Hybrid Search
They have some off the shelf functionalities and a way to implement the custom Re-Ranking and Filtering Function here [Implement Custom Rerankers](https://lancedb.github.io/lancedb/hybrid_search/#building-custom-rerankers)

In [ ]:
from lancedb.rerankers import LinearCombinationReranker

reranker = LinearCombinationReranker(weight=0.7) # Weight = 0 Means pure Text Search (BM-25) and 1 means pure Sementic (Vector) Search

table.search("To confuse the AI and DNN embedding, let's put random terms from other sentences- automation training test memory?", query_type="hybrid").\
            rerank(reranker=reranker).\
            limit(5).\
            to_pandas()

,vector,text,title,num_words,_relevance_score
0,"[-0.029037403, -0.0019001433, 0.01014971, -0.0...",Classifying short texts to one category or clu...,Using deep learning for short text understanding,187,0.700000
1,"[-0.013957698, 0.02428029, 0.0018158188, -0.03...",Hierarchical Pitman-Yor Process priors are com...,Compressed Nonparametric Language Modelling,134,0.674416
2,"[-0.013545574, -0.0019434954, 0.009623939, -0....",a r t i c l e i n f o a b s t r a c t We prese...,"BabelNet: The automatic construction, evaluati...",133,0.256456
3,"[-0.035857946, -0.00020074612, 0.009314451, -0...",We introduce a long short-term memory recurren...,Speech-driven 3 D Facial Animation with Implic...,171,0.123807
4,"[-0.027462298, -0.00018946378, 0.017612722, -0...",We introduce a technique for augmenting neural...,Deep Voice 2 : Multi-Speaker Neural Text-to-Sp...,151,0.112552


## Build custom Filtering Function

By passing the `pandas.query` style, filtering, we will do the following 2 things:

1. Remove all the rows which contain a specific term, in out case, `"dual-band"`
2. Keep only the rows which have `num_words > 100`

In [ ]:
from typing import List, Union
import pandas as pd
import pyarrow as pa

class MofidifiedLinearReranker(LinearCombinationReranker):
    def __init__(self, filters: Union[str, List[str]], **kwargs):
        super().__init__(**kwargs)
        filters = filters if isinstance(filters, list) else [filters]
        self.filters = filters

    def rerank_hybrid(self, query: str, vector_results: pa.Table, fts_results: pa.Table)-> pa.Table:
        combined_result = super().rerank_hybrid(query, vector_results, fts_results)
        df = combined_result.to_pandas()
        for filter in self.filters:
            df = df.query("(not text.str.contains(@filter)) & (num_words > 150) ") # THIS is where you implement your filters. You can hard code or pass dynamically too

        return pa.Table.from_pandas(df)

modified_reranker = MofidifiedLinearReranker(filters=["dual-band"])

table.search("To confuse the AI and DNN embedding, let's put random terms from other sentences- automation training test memory?", query_type="hybrid").\
            rerank(reranker=modified_reranker).\
            limit(7).\
            to_pandas()

,vector,text,title,num_words,_relevance_score
0,"[-0.029037403, -0.0019001433, 0.01014971, -0.0...",Classifying short texts to one category or clu...,Using deep learning for short text understanding,187,0.700000
3,"[-0.035857946, -0.00020074612, 0.009314451, -0...",We introduce a long short-term memory recurren...,Speech-driven 3 D Facial Animation with Implic...,171,0.298656
4,"[-0.027462298, -0.00018946378, 0.017612722, -0...",We introduce a technique for augmenting neural...,Deep Voice 2 : Multi-Speaker Neural Text-to-Sp...,151,0.209681
5,"[0.0012136719, -0.022606706, 0.0142465895, -0....",Software organizations have typically de-empha...,A survey of software testing practices in alberta,236,0.198860
7,"[-0.027905056, -4.77228e-05, 0.0129647385, -0....",In this paper a novel approach is proposed to ...,Text mining of news-headlines for FOREX market...,621,0.100694
